In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Corona_NLP.csv')

In [ ]:
df.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()

In [ ]:
df['label'] = encoder.fit_transform(df['Sentiment'])

In [ ]:
df.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
df.drop(columns=['Sentiment'], axis=1, inplace=True)

In [ ]:
df.head()

,OriginalTweet,label
0,TRENDING: New Yorkers encounter empty supermar...,0
1,When I couldn't find hand sanitizer at Fred Me...,4
2,Find out how you can protect yourself and love...,1
3,#Panic buying hits #NewYork City as anxious sh...,2
4,#toiletpaper #dunnypaper #coronavirus #coronav...,3


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train,df_test=train_test_split(df, train_size=0.2, random_state=42)

In [ ]:
df_train.to_csv('train.csv')
df_test.to_csv('test.csv')

In [ ]:
!pip install -U accelerate
!pip install -U transformers[torch]
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("csv",data_files={'train':'train.csv', 'test':'test.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
encoded_dataset = dataset.map(lambda t: tokenizer(t['OriginalTweet'],  truncation=True), batched=True,load_from_cache_file=False)


Map:   0%|          | 0/759 [00:00<?, ? examples/s]

Map:   0%|          | 0/3039 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels =5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
arg = TrainingArguments(
    "sentiment",
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    seed=19
)


In [ ]:
trainer = Trainer(
    model=model,
    args=arg,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=95, training_loss=1.5994779887952302, metrics={'train_runtime': 17.2834, 'train_samples_per_second': 43.915, 'train_steps_per_second': 5.497, 'total_flos': 32506910354610.0, 'train_loss': 1.5994779887952302, 'epoch': 1.0})

In [ ]:
y_pred = trainer.predict(encoded_dataset['test'])

In [ ]:
y_pred

PredictionOutput(predictions=array([[-0.6430133 , -0.43816018,  0.5887605 ,  0.57909894,  0.17947379],
       [-0.37234658, -0.31597126,  0.28108343, -0.23592825, -0.04381441],
       [-0.64458984, -0.47143933,  0.3961855 ,  0.27716243,  0.22120455],
       ...,
       [-0.6511898 , -0.5239657 ,  0.4422344 ,  0.43358842,  0.24057096],
       [-0.5913537 , -0.5035456 ,  0.4164593 ,  0.27268192,  0.21173644],
       [-0.31326008, -0.43164617,  0.23814684, -0.11071958,  0.05480059]],
      dtype=float32), label_ids=array([3, 0, 0, ..., 3, 3, 1]), metrics={'test_loss': 1.5496702194213867, 'test_runtime': 18.947, 'test_samples_per_second': 160.395, 'test_steps_per_second': 20.056})

In [ ]:
y_pred = y_pred.predictions

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

In [ ]:
np.argmax(y_pred[0])

2

In [ ]:
y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]

In [ ]:
f1_score(df_test['label'],y_pred,average='macro')

0.10292959273220315